In [1]:
# note to self :
# use rdkit for 2d reps
# use jacques' code for sdf -> xyz

In [ ]:
# from pyscf import gto
import os
import numpy as np
from rdkit import Chem
from rdkit.Chem import SDWriter
import glob
import qml2
from qml2.representations.slatm import get_slatm_mbtypes, generate_slatm
from qstack.spahm.compute_spahm import get_spahm_representation
from qstack.compound import xyz_to_mol
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys




In [ ]:
aa_onelettercode = ["A",
                "C",
                "D",
                "E",
                "F",
                "G",
                "H",
                "I",
                "K",
                "L",
                "M",
                "N",
                "P",
                "Q",
                "R",
                "S",
                "T",
                "V",
                "W",
                "Y"]

aa_smiles = {
    "A": "C[C@@H](C(=O)O)N",
    "C": "C([C@@H](C(=O)O)N)S",
    "D": "C([C@@H](C(=O)O)N)C(=O)O",
    "E": "C(CC(=O)O)[C@@H](C(=O)O)N",
    "F": "C1=CC=C(C=C1)C[C@@H](C(=O)O)N",
    "G": "C(C(=O)O)N",
    "H": "C1=C(NC=N1)C[C@@H](C(=O)O)N",
    "I": "CC[C@H](C)[C@@H](C(=O)O)N",
    "K": "C(CCN)C[C@@H](C(=O)O)N",
    "L": "CC(C)C[C@@H](C(=O)O)N",
    "M": "CSCC[C@@H](C(=O)O)N",
    "N": "C([C@@H](C(=O)O)N)C(=O)N",
    "P": "C1C[C@H](NC1)C(=O)O",
    "Q": "C(CC(=O)N)[C@@H](C(=O)O)N",
    "R": "C(C[C@@H](C(=O)O)N)CN=C(N)N",
    "S": "C([C@@H](C(=O)O)N)O",
    "T": "C[C@H]([C@@H](C(=O)O)N)O",
    "V": "CC(C)[C@@H](C(=O)O)N",
    "W": "C1=CC=C2C(=C1)C(=CN2)C[C@@H](C(=O)O)N",
    "Y": "C1=CC(=CC=C1C[C@@H](C(=O)O)N)O"
}

Import the smiles of the amino acids

Convert the sdf files in xyz files

In [ ]:
def sdf_to_xyz(input_dir, output_dir):
    """
    Converts .sdf files in .xyz files and stores them in a new directory
    Important : each file contains only one molecule
    inputs :
        input_dir : str, name of directory where .sdf files are stored
        output_dir : str, name of directory where .xyz files will be stored
    returns :
        
    """
    if output_dir not in os.listdir("."):
        os.mkdir(output_dir)
    sdf_files = glob.glob(input_dir + '/*.sdf')

    for i, sdf_file in enumerate(sdf_files):
        suppl = Chem.rdmolfiles.ForwardSDMolSupplier(sdf_file, removeHs=False)
        label = aa_onelettercode[i]
        for mol in suppl:
            Chem.rdmolfiles.MolToXYZFile(mol, output_dir + "/" + label + ".xyz")

In [ ]:
sdf_to_xyz("3d_struct_aa_sdf", "3d_struct_aa_xyz")

Create the list of 20 sublists containing the chemical info of the amino acids

In [ ]:
def create_aa_list(input_dir):
    """
    Creates a list of 20 sublists of type [qml.Compound, n_atoms] from 20 xyz files
    inputs : 
        input_dir : str, name of directory where .xyz files are stored
    returns :
        compoundsList : list of 20 sublists [qml.Compound, n_atoms] with the info of each aa
    """
    xyz_files = glob.glob(input_dir + '/*.xyz')
    compoundsList = []
    for xyz_file in xyz_files:
        mol = qml2.Compound(input_dir + "/" + xyz_file)
        
        with open(input_dir + "/" + xyz_file, 'r') as file:
            lines = file.read().split('\n')
            numAtoms = int(lines[0])
        
        subList = [mol, numAtoms]
        compoundsList.append(subList)

    return compoundsList


Generates the representation matrices of each representation : one-hot, SLATM, SPAHM, Morgan FP and MACCS Keys

In [ ]:


class RepresentationGenerator:
    """
    This object serves to generate a given representation for a set of molecules
    To generate a given representation, initialize a RepresentationGenerator object (argument: "cMBDF", "SLATM", or "Coulomb", representation that is going to be generated)
    A list of sublists of the type [qml.Compound, number_of_atoms] must be given
    In any case, the generation of a representation gives as an output a numpy array, each row represents a single molecule
    """
    def __init__(self, repType):
        self.repType = repType
        self.repDict = {
            "cMBDF": self.cMBDFRepresentation,
            "SLATM": self.SLATMRepresentation,
            "Coulomb": self.CoulombRepresentation,
        }
    
    def OneHotRepresentation(self):
        """
        Generates the one-hot representation of each amino-acid
        inputs:
            -
        returns:
            one_hot : 21 x 21 identity matrix
        """
        labels = np.arange(1, 22, 1)
        unique_labels, indices = np.unique(labels, return_inverse=True)
        one_hot = np.eye(len(unique_labels))[indices]
        return one_hot

    
    def SLATMRepresentation(self, list_XYZ):
        compoundsList = [subList[0] for subList in list_XYZ]

        charges = [compound.nuclear_charges for compound in compoundsList]
        coordinates = [compound.coordinates for compound in compoundsList]

        mbtypes = get_slatm_mbtypes(charges)
        repsList = []
        print("Generating SLATM representations: ")
        for i in range(len(compoundsList)):
            rep = generate_slatm(charges[i], coordinates[i], mbtypes)
            repsList.append(rep)

        A = np.row_stack(tuple(repsList))
        return A
    
    def SPAHMRepresentation(self, list_xyz):
        # TODO
        # use xyz_to_mol(filename)
        # use get_spahm_representation(mol, guess_in, xc="pbe")
        return A

    def MorganFPRepresentation(self, list_smiles):
        # TODO
        # use mol = Chem.MolFromSmiles(smiles)
        # use fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return A
    
    def MACCSKeysRepresentation(self, list_smiles):
        # TODO
        # use mol = Chem.MolFromSmiles(smiles)
        # use fingerprint = MACCSkeys.GenMACCSKeys(mol)
        return A
    
    # def CoulombRepresentation(self, list_XYZ):
    #     numCompounds = len(list_XYZ)
    #     lenList = [subList[1] for subList in list_XYZ]
    #     lMax = max(lenList)

    #     for subList in list_XYZ:
    #         subList[0].generate_coulomb_matrix(size=lMax, sorting="row-norm")

    #     l = len(subList[0].representation)
    #     A = np.zeros((numCompounds, l), dtype=float)

    #     for i, subList in enumerate(list_XYZ):
    #         for j, val in enumerate(subList[0].representation):
    #             A[i, j] = val

    #     return A
    
    def RepresentationGeneration(self, molsList):
        A = self.repDict[self.repType](molsList)
        return A

    